<a href="https://colab.research.google.com/github/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [1]:
# Write your code here
# open and read file
import nltk
nltk.download('punkt')
with open("cleaned_review_content_forAssign3.txt", "r", encoding='utf-8') as file:
  data = file.read()




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# (1) Count the frequency of all the N-grams (N=3)
import pandas
from textblob import TextBlob
tri_grams=TextBlob(data).ngrams(3)
#print(tri_grams)
hist=dict()
for gram in tri_grams:
  gram=tuple(gram)
  hist[gram]=hist.get(gram,0)+1

pandas.DataFrame.from_dict(hist, orient = 'index', columns=['tri-grams frequency'])



,tri-grams frequency
"(person, saw, hype)",1
"(saw, hype, claim)",1
"(hype, claim, masterpiec)",1
"(claim, masterpiec, overreact)",1
"(masterpiec, overreact, overblown)",1
...,...
"(mayb, famili, friend)",1
"(famili, friend, peopl)",1
"(friend, peopl, work)",1
"(peopl, work, movi)",1


In [5]:
#(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.
import pandas
monty = TextBlob (data)
bi_grams=TextBlob(data).ngrams(2)
#print(bi_grams)
hist={}
for gram in bi_grams:
  gram=tuple(gram)
  hist[gram]=hist.get(gram,0)+1

bigr_prob={}
for key, val in hist.items():
  bigr_prob[key]=val/monty.words.count(key[0])
   
pandas.DataFrame.from_dict(bigr_prob, orient = 'index', columns=['bigrams probabilities']).head(10)





,bigrams probabilities
"(person, saw)",0.052632
"(saw, hype)",0.076923
"(hype, claim)",0.066667
"(claim, masterpiec)",0.500000
"(masterpiec, overreact)",0.047619
"(overreact, overblown)",1.000000
"(overblown, excit)",1.000000
"(excit, anoth)",0.500000
"(anoth, joker)",0.083333
"(joker, base)",0.004762


In [7]:
#(3) Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).
# Extract all the noun phrases 
import pandas
import nltk
nltk.download('brown')
monty=TextBlob(data)
print(monty.noun_phrases)

hist={}
for np in monty.noun_phrases:
  hist[np]=hist.get(np,0)+1

max_refreq = max([val for val in hist.values()])

re_prob={} 
for key, val in hist.items():
  re_prob[key] = val/max_refreq

pandas.DataFrame.from_dict (re_prob, orient = 'index', columns = ['noun-phrases relative probabilities']).head(10)





[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
['hype claim masterpiec overreact overblown excit anoth joker base film', 'bit pretenti trailer', 'wrong massiv achiev cinema', 'rare day age cgi nonsens reboot', 'reboot sort standalon origin tale impecc', 'finish echo resembl', 'joker origin', 'comic past joaquin', 'sweat cri everi drop magnific dedic', 'heath ledger', 'proud undoubtedli', 'sinc heath joker', 'brilliant bleak set tone palpabl', 'film place blown', 'everi audienc member awestruck wit film', 'transport charact world', 'believ hype', 'rever transcend masterpiec cinema everi movi', 'impact joaquin', 'scenographi brillianc grotesqu haunt cringi', 'time mesmer wont blink eye', 'serious funni moment emot rollercoast sometim multipl emot poppingup timethi', 'typic actionriddl', 'superhero movi', 'proper psycholog thrillerdrama singl', 'alon isol truli relat', 'sorri charact lot peopl', 'encourag violenc truli movi e

,noun-phrases relative probabilities
hype claim masterpiec overreact overblown excit anoth joker base film,0.2
bit pretenti trailer,0.2
wrong massiv achiev cinema,0.2
rare day age cgi nonsens reboot,0.2
reboot sort standalon origin tale impecc,0.2
finish echo resembl,0.2
joker origin,0.2
comic past joaquin,0.2
sweat cri everi drop magnific dedic,0.2
heath ledger,0.4


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [12]:
# (1) To build the documents-terms weights (tf*idf) matrix bold text.
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
voca=[word for word in set(data.split())]
corpus = {v: k for v, k in enumerate(data.splitlines(),1)}
tfidf=TfidfVectorizer(vocabulary=voca, max_features=50, lowercase=True, analyzer='word', stop_words='english', ngram_range=(1,3))
tfs=tfidf.fit_transform(corpus.values())
feature_names= tfidf.get_feature_names()
corpus_index=[n for n in corpus]
df=pandas.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
df.head(10)



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111
longerwhich,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.179083,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
mogul,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.072762,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
set,0.085715,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.073125,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.118013,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.050984,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.172057,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.044577,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
applaus,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.051498,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.239464,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
standalon,0.105454,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.076215,0.0,0.0,0.0,0.0,0.0,0.096865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
dozen,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.079525,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
main,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.141874,0.0,0.059011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.038077,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.108546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.060095,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068519,0.0,0.0,0.0,0.0
funni,0.000000,0.161832,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.108916,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
oscarwin,0.000000,0.000000,0.0,0.000000,0.0,0.177326,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000

In [26]:
#(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using cosine similarity.
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
from textblob import Word
query="I'm not a big fan of action movie, but this movie is REALLY a masterpiece!!! It is a perfect combination of acting, storyline, soundtrack and everything. It deserves an Oscar of this year."
# preprocess query
query=query.lower()
query=re.sub('[^\w\s]', '', query)
stop=stopwords.words('english')
query= " ".join(x for x in query.split() if x not in stop)
st=PorterStemmer()
query=" ".join([st.stem(word) for word in query.split()])
query= " ".join([Word(word).lemmatize() for word in query.split()])
print(query)

# add query to review corpus
with open("reviews_query_forAssign3.txt", "a", encoding='utf-8') as file:
  file.write(data)
  #file.write('\n')
  file.write(query)
with open("reviews_query_forAssign3.txt", "r", encoding='utf-8') as file:
  data_query = file.read()






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
im big fan action movi movi realli masterpiec perfect combin act storylin soundtrack everyth deserv oscar year


In [28]:
# extract features representing all reviews and the query
voca=[word for word in set(data_query.split())]
corpus = {v: k for v, k in enumerate(data_query.splitlines(),1)}
tfidf=TfidfVectorizer(vocabulary=voca, max_features=50, lowercase=True, analyzer='word', stop_words='english', ngram_range=(1,3))
tfs=tfidf.fit_transform(corpus.values())
feature_names= tfidf.get_feature_names()
#print(len(feature_names))
corpus_index=[n for n in corpus]
df=pandas.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112
longerwhich,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.178969,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
mogul,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.072734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
set,0.085768,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.073158,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.118301,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.051003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.172163,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.044597,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
applaus,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.051502,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.239574,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
standalon,0.105470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.076201,0.000000,0.0,0.0,0.0,0.0,0.096851,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
teeter,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.058476,0.0,0.000000,0.053662,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
stuff,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.056014,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
flavour,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.058362,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
stay,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000

In [65]:
import pandas
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

i=1
quer_sim = {}
while i <=111:
  cosim=cosine_similarity(tfs[111], tfs[i])
  quer_sim['doc'+str(i)] = cosim
  i+=1
print(quer_sim)
sorted (quer_sim, key=quer_sim.get, reverse=True)
quer_sim_rank = {key:rank for rank, key in enumerate(sorted (quer_sim, key=quer_sim.get, reverse=True),1)}
rankdf=pandas.DataFrame.from_dict (quer_sim_rank,orient = 'index', columns = ['ranking']).head(10)
print(rankdf)
print(quer_sim['doc81'])
print(data_query.splitlines()[81])
print(data_query.splitlines()[111])


{'doc1': array([[0.01903672]]), 'doc2': array([[0.03117548]]), 'doc3': array([[0.05510742]]), 'doc4': array([[0.05440095]]), 'doc5': array([[0.]]), 'doc6': array([[0.06801917]]), 'doc7': array([[0.05958245]]), 'doc8': array([[0.06118135]]), 'doc9': array([[0.15154574]]), 'doc10': array([[0.06437229]]), 'doc11': array([[0.0573967]]), 'doc12': array([[0.03706264]]), 'doc13': array([[0.14005546]]), 'doc14': array([[0.0247722]]), 'doc15': array([[0.12676382]]), 'doc16': array([[0.10853458]]), 'doc17': array([[0.09130311]]), 'doc18': array([[0.04357387]]), 'doc19': array([[0.19674455]]), 'doc20': array([[0.02540792]]), 'doc21': array([[0.0948607]]), 'doc22': array([[0.07539785]]), 'doc23': array([[0.01815613]]), 'doc24': array([[0.07201808]]), 'doc25': array([[0.06957643]]), 'doc26': array([[0.16581357]]), 'doc27': array([[0.02100748]]), 'doc28': array([[0.09458971]]), 'doc29': array([[0.07814741]]), 'doc30': array([[0.09971216]]), 'doc31': array([[0.]]), 'doc32': array([[0.04091819]]), 'do

[3 1 0 2]
[0 1 2 3]


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: 
https://github.com/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/Huyen_labeled_review_sentiments.csv 
